In [24]:
'''import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Load the data
df = pd.read_csv('/Users/chetan/Documents/GitHub/nj_transit_data_ru_hack/data/rail_data/2020_05.csv')  # Replace with your actual file name

# Convert date and time columns to datetime
df['date'] = pd.to_datetime(df['date'])
df['scheduled_time'] = pd.to_datetime(df['scheduled_time'])
df['actual_time'] = pd.to_datetime(df['actual_time'])

# Calculate delay in minutes
df['delay_minutes'] = (df['actual_time'] - df['scheduled_time']).dt.total_seconds() / 60

# Basic statistics
print("Basic Statistics:")
print(df['delay_minutes'].describe())

# Average delay by line
avg_delay_by_line = df.groupby('line')['delay_minutes'].mean().sort_values(ascending=False)
print("\nAverage Delay by Line:")
print(avg_delay_by_line)

# Count of trips by status
status_counts = df['status'].value_counts()
print("\nTrip Status Counts:")
print(status_counts)

# Visualizations
plt.figure(figsize=(12, 6))
avg_delay_by_line.plot(kind='bar')
plt.title('Average Delay by Line')
plt.xlabel('Line')
plt.ylabel('Average Delay (minutes)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('avg_delay_by_line.png')

plt.figure(figsize=(10, 6))
status_counts.plot(kind='pie', autopct='%1.1f%%')
plt.title('Trip Status Distribution')
plt.axis('equal')
plt.savefig('trip_status_distribution.png')

# On-time performance
on_time_threshold = 5  # Consider trips with delay <= 5 minutes as on-time
df['on_time'] = df['delay_minutes'] <= on_time_threshold
on_time_performance = df['on_time'].mean() * 100

print(f"\nOn-Time Performance: {on_time_performance:.2f}%")

# Busiest stations
busiest_stations = df['from'].value_counts().head(10)
print("\nTop 10 Busiest Stations:")
print(busiest_stations)

# Save processed data
df.to_csv('processed_transit_data.csv', index=False)
print("\nProcessed data saved to 'processed_transit_data.csv'")

# Additional analysis: Delay patterns by time of day
df['hour'] = df['scheduled_time'].dt.hour
hourly_delays = df.groupby('hour')['delay_minutes'].mean()

plt.figure(figsize=(12, 6))
hourly_delays.plot(kind='line', marker='o')
plt.title('Average Delay by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Delay (minutes)')
plt.xticks(range(0, 24))
plt.grid(True)
plt.savefig('delay_by_hour.png')

print("\nAnalysis complete. Check the generated CSV file and PNG images for results.")'''

'import pandas as pd\nimport matplotlib.pyplot as plt\nfrom datetime import datetime\n\n# Load the data\ndf = pd.read_csv(\'/Users/chetan/Documents/GitHub/nj_transit_data_ru_hack/data/rail_data/2020_05.csv\')  # Replace with your actual file name\n\n# Convert date and time columns to datetime\ndf[\'date\'] = pd.to_datetime(df[\'date\'])\ndf[\'scheduled_time\'] = pd.to_datetime(df[\'scheduled_time\'])\ndf[\'actual_time\'] = pd.to_datetime(df[\'actual_time\'])\n\n# Calculate delay in minutes\ndf[\'delay_minutes\'] = (df[\'actual_time\'] - df[\'scheduled_time\']).dt.total_seconds() / 60\n\n# Basic statistics\nprint("Basic Statistics:")\nprint(df[\'delay_minutes\'].describe())\n\n# Average delay by line\navg_delay_by_line = df.groupby(\'line\')[\'delay_minutes\'].mean().sort_values(ascending=False)\nprint("\nAverage Delay by Line:")\nprint(avg_delay_by_line)\n\n# Count of trips by status\nstatus_counts = df[\'status\'].value_counts()\nprint("\nTrip Status Counts:")\nprint(status_counts)\n\

In [25]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load Data
data_path = '/Users/chetan/Documents/GitHub/nj_transit_data_ru_hack/data/rail_data/2020_05.csv'  # Update to your data path
df = pd.read_csv(data_path)

# Display the first few rows of the data
print("Train Schedule Data:")
print(df.head())

# Step 2: Data Preprocessing
# Convert date columns to datetime format
df['scheduled_time'] = pd.to_datetime(df['scheduled_time'])
df['actual_time'] = pd.to_datetime(df['actual_time'])

# Calculate delay in minutes if not already in minutes
df['delay_minutes'] = df['delay_minutes'].fillna(0)  # Fill NaN with 0 for delays

# Extract additional time features
df['hour_of_day'] = df['scheduled_time'].dt.hour
df['day_of_week'] = df['scheduled_time'].dt.dayofweek

# Step 3: Prepare Features and Target


'''for col in ['from_id', 'to_id']:
    freq = df[col].value_counts() / len(df)
    df[col + '_freq_encoded'] = df[col].map(freq)

# Drop the original columns if needed
df = df.drop(columns=['from_id', 'to_id'])
features = ['hour_of_day', 'day_of_week', 'train_id', 'from_id_freq_encoded', 'to_id_freq_encoded']  # Select relevant features'''
features = ['hour_of_day', 'day_of_week', 'train_id', 'from_id', 'to_id']  # Select relevant features
target = 'delay_minutes'  # We want to predict delay minutes

# Prepare data for training
X = df[features]
y = df[target]

# Check for NaN values and drop them if necessary
X = X.dropna()
y = y[X.index]  # Ensure target matches features
print(X.head())


Train Schedule Data:
         date train_id  stop_sequence                 from  from_id  \
0  2020-05-01     5543            1.0  Newark Penn Station      107   
1  2020-05-01     5543            2.0  Newark Penn Station      107   
2  2020-05-01     5543            3.0                Union    38105   
3  2020-05-01     5543            4.0         Roselle Park       31   
4  2020-05-01     5543            5.0             Cranford       32   

                    to  to_id       scheduled_time          actual_time  \
0  Newark Penn Station    107  2020-05-01 23:38:00  2020-05-01 23:40:09   
1                Union  38105  2020-05-01 23:47:00  2020-05-01 23:47:01   
2         Roselle Park     31  2020-05-01 23:50:00  2020-05-01 23:51:04   
3             Cranford     32  2020-05-01 23:55:00  2020-05-01 23:55:31   
4            Westfield    155  2020-05-01 23:59:00  2020-05-01 23:59:01   

   delay_minutes    status            line        type  
0       2.150000  departed  Raritan Valley  

In [26]:
# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [27]:
# Step 5: Initialize and train the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)



RandomForestRegressor(random_state=42)

In [28]:
# Step 6: Predictions and Evaluation
y_pred = model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nModel Performance:")
print(f"Mean Absolute Error (MAE): {mae:.2f} minutes")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} minutes")




Model Performance:
Mean Absolute Error (MAE): 1.63 minutes
Root Mean Squared Error (RMSE): 3.60 minutes


In [29]:
# Step 7: Prediction Function
def predict_delay(hour_of_day, day_of_week, train_id, from_id, to_id):
    # Create a feature vector
    input_data = pd.DataFrame([{
        'hour_of_day': hour_of_day,
        'day_of_week': day_of_week,
        'train_id': train_id,
        'from_id': from_id,
        'to_id': to_id
    }])

    
    # Align the input data with the training data columns
    for column in X.columns:
        if column not in input_data.columns:
            input_data[column] = 0  # Add missing columns with default value 0
    
    input_data = input_data[X.columns]  # Align column order

    # Predict delay
    predicted_delay = model.predict(input_data)[0]
    return predicted_delay



In [30]:
# Example prediction
predicted = predict_delay(18, 6, 3805, 105, 31)  # Example values
print(f"\nPredicted Delay (minutes): {predicted:.2f}")


Predicted Delay (minutes): 1.79
